In [1]:
PATH = "/home/thesis/"
# PATH = "/dss/dsshome1/0A/di93hoq/"

In [4]:
import scanpy as sc
import numpy as np
import os
from pathlib import Path
from pprint import pprint
from seml.config import generate_configs, read_config
from chemCPA.experiments_run import ExperimentWrapper

In [5]:
os.chdir(PATH + "ConditionalMongeGap/")
from losses import sinkhorn_div

In [3]:
os.getcwd()

'/dss/dsshome1/0A/di93hoq/forkCPA/compare'

In [4]:
# Load the dataset
adata = sc.read_h5ad(PATH + "/ConditionalMongeGap/Datasets/sciplex_complete_middle_subset.h5ad")

In [5]:
adata = adata[
    (adata.obs["cell_type"] == "A549")
    & ((adata.obs["dose"] == 10000) | (adata.obs["dose"] == 0))
]

In [7]:
# Save the dataset
adata.write_h5ad(PATH + "ConditionalMongeGap/Datasets/sciplex_complete_middle_subset_compare.h5ad")

In [8]:
os.chdir(PATH + "forkCPA/")

/dss/dsshome1/0A/di93hoq/forkCPA


In [9]:
exp = ExperimentWrapper(init_all=False)
# this is how seml loads the config file internally
assert Path(
    "manual_run.yaml"
).exists(), "config file not found"
seml_config, slurm_config, experiment_config = read_config(
   "manual_run.yaml"
)
# we take the first config generated
configs = generate_configs(experiment_config)
if len(configs) > 1:
    print("Careful, more than one config generated from the yaml file")
args = configs[0]
pprint(args)

exp.seed = 1337
# loads the dataset splits
exp.init_dataset(**args["dataset"])

{'dataset': {'data_params': {'covariate_keys': 'cell_type',
                             'dataset_path': '/dss/dsshome1/0A/di93hoq/ConditionalMongeGap/Datasets/sciplex_complete_middle_subset_compare.h5ad',
                             'degs_key': 'all_DEGs',
                             'dose_key': 'dose',
                             'pert_category': 'cov_drug_dose_name',
                             'perturbation_key': 'condition',
                             'smiles_key': 'SMILES',
                             'split_key': 'split_ood_finetuning',
                             'use_drugs_idx': True},
             'dataset_type': 'trapnell'},
 'model': {'additional_params': {'decoder_activation': 'ReLU',
                                 'doser_type': 'amortized',
                                 'multi_task': False,
                                 'patience': 50,
                                 'seed': 1337},
           'append_ae_layer': True,
           'embedding': {'directory': 

In [10]:
exp.init_drug_embedding(embedding=args["model"]["embedding"])
exp.init_model(
    hparams=args["model"]["hparams"],
    additional_params=args["model"]["additional_params"],
    load_pretrained=args["model"]["load_pretrained"],
    append_ae_layer=args["model"]["append_ae_layer"],
    enable_cpa_mode=args["model"]["enable_cpa_mode"],
    pretrained_model_path=args["model"]["pretrained_model_path"],
    pretrained_model_hashes=args["model"]["pretrained_model_hashes"],
)

In [11]:
exp.datasets.keys()

dict_keys(['training', 'training_control', 'training_treated', 'test', 'test_control', 'test_treated', 'ood'])

In [12]:
# setup the torch DataLoader
exp.update_datasets()

In [13]:
exp.train(**args["training"])

CWD: /dss/dsshome1/0A/di93hoq/forkCPA
Save dir: compare/checkpoints
Size of disentanglement testdata: 2667

Took 1.5 min for evaluation.



{'epoch': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36],
 'stats_epoch': [36],
 'loss_reconstruction': [-278.66813130676746,
  -490.9462865740061,
  -515.5409076213837,
  -553.6104661822319,
  -563.2690917849541,
  -579.3865351080894,
  -601.3836835026741,
  -611.4821467399597,
  -616.7225347459316,
  -622.3780278116465,
  -625.1538212895393,
  -626.7321664690971,
  -629.1828764528036,
  -628.9401694089174,
  -620.441006064415,
  -638.9769082665443,
  -625.7234669029713,
  -642.139610901475,
  -645.9147500991821,
  -651.2050248384476,
  -653.9029611051083,
  -621.4622209817171,
  -633.6607711911201,
  -645.3905718326569,
  -641.8460663557053,
  -655.5221939086914,
  -654.988538146019,
  -660.0596520900726,
  -662.2556347846985,
  -647.3766260147095,
  -652.9646739959717,
  -656.1138916015625,
  -656.5170629024506,
  -658.81

In [14]:
!ls compare/checkpoints

model_36.pt  model_4.pt


# Model loading and preparation

In [23]:
os.getcwd()

'/dss/dsshome1/0A/di93hoq/forkCPA'

In [24]:
from notebooks.utils import load_config, load_dataset, load_model, load_smiles, compute_pred
from chemCPA.train import evaluate, compute_prediction, repeat_n
import torch
from chemCPA.model import ComPert

In [25]:
seml_collection = "multi_task"
model_hash_pretrained_rdkit = "model_4"  # Fine-tuned

In [26]:
model = torch.load("compare/checkpoints/model_36.pt")

In [27]:
(
    state_dict,
    cov_adv_state_dicts,
    cov_emb_state_dicts,
    init_args,
    history,
        ) = model

In [28]:
model = ComPert(
        **init_args, drug_embeddings=exp.drug_embeddings
)
model = model.eval()

In [29]:
drug_r2_pretrained_degs_rdkit, _ = compute_pred(
    model,
    exp.datasets["ood"]
)

['A549', 'K562', 'MCF7']


9it [00:00, 50.34it/s]

A549_CUDC-101_1.0: -0.77
A549_CUDC-907_1.0: -0.41
A549_Dacinostat_1.0: -0.58
A549_Givinostat_1.0: -1.31
A549_Hesperadin_1.0: -0.44
A549_Pirarubicin_1.0: -0.46
A549_Raltitrexed_1.0: -1.64
A549_Tanespimycin_1.0: -1.09
A549_Trametinib_1.0: -0.56


In [31]:
model.predict(
    genes=exp.datasets["ood"].genes,
    drugs_idx=exp.datasets["ood"].drugs_idx,
    dosages=exp.datasets["ood"].dosages,
    covariates=exp.datasets["ood"].covariates
)

(tensor([[0.0366, 0.0855, 0.0778,  ..., 0.7143, 0.6959, 0.6535],
         [0.0368, 0.0855, 0.0777,  ..., 0.7143, 0.6959, 0.6536],
         [0.0365, 0.0857, 0.0777,  ..., 0.7142, 0.6959, 0.6535],
         ...,
         [0.0368, 0.0855, 0.0778,  ..., 0.7142, 0.6959, 0.6536],
         [0.0367, 0.0853, 0.0778,  ..., 0.7142, 0.6958, 0.6536],
         [0.0366, 0.0852, 0.0780,  ..., 0.7145, 0.6959, 0.6537]],
        grad_fn=<CatBackward0>),
 tensor([[-1.4938,  0.8747,  0.9239,  ..., -0.0722, -0.0473,  0.0108],
         [-1.4938,  0.8747,  0.9239,  ..., -0.0642, -0.0438,  0.0150],
         [-1.4938,  0.8747,  0.9239,  ..., -0.0625, -0.0356,  0.0140],
         ...,
         [-1.4938,  0.8747,  0.9239,  ..., -0.0642, -0.0438,  0.0150],
         [-1.4938,  0.8747,  0.9239,  ..., -0.0670, -0.0402,  0.0243],
         [-1.4938,  0.8747,  0.9239,  ..., -0.0520, -0.0328,  0.0136]],
        grad_fn=<CatBackward0>))